# Process images in `./imgs/`

## Object Detection Models
- [Rex-Omni](https://github.com/IDEA-Research/Rex-Omni) ❌ can't install
- YOLO [11](https://huggingface.co/Ultralytics/YOLO11) ❌ [uses COCO classes](https://docs.ultralytics.com/tasks/detect/#models): no trees, [just person](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco.yaml)
- DETR [resnet-101](https://huggingface.co/facebook/detr-resnet-101) ❌ uses COCO classes: no trees, just person
- OWLv2 [large](https://huggingface.co/google/owlv2-large-patch14-ensemble) ✅ ok with IOU filtering
- Grounding DINO [base](https://huggingface.co/IDEA-Research/grounding-dino-base) ✅ best with IOU filtering

## Object Segmentation Models
- [SAM3](https://ai.meta.com/blog/segment-anything-model-3/)
- SegFormer [[b5-ade](https://huggingface.co/nvidia/segformer-b5-finetuned-ade-640-640)] (persons, trees)
- DETR [[resnet-50-panoptic](https://huggingface.co/facebook/detr-resnet-50-panoptic)] (persons, trees)
- MaskFormer [[swin-base-ade](https://huggingface.co/facebook/maskformer-swin-base-ade)] (persons, trees)

## TODO:

### Detect shade pixels (mask)

### Detect actual bus stop

In [ ]:
!pip install ultralytics

In [ ]:
!wget -q https://raw.githubusercontent.com/direito-a-sombra/bus-view/refs/heads/main/models/detect_utils.py
!wget -q https://raw.githubusercontent.com/direito-a-sombra/bus-view/refs/heads/main/models/Detr.py
!wget -q https://raw.githubusercontent.com/direito-a-sombra/bus-view/refs/heads/main/models/Dino.py
!wget -q https://raw.githubusercontent.com/direito-a-sombra/bus-view/refs/heads/main/models/Owlv2.py
!wget -q https://raw.githubusercontent.com/direito-a-sombra/bus-view/refs/heads/main/models/Yolo.py
!mkdir -p ./models && mv *.py ./models

!wget -q https://github.com/direito-a-sombra/bus-view/releases/latest/download/imgs.tar.gz
!wget -q https://raw.githubusercontent.com/direito-a-sombra/bus-view/refs/heads/main/data/training/train_boxes.json
!wget -q https://raw.githubusercontent.com/direito-a-sombra/bus-view/refs/heads/main/data/training/train_files.json
!tar -xzf imgs.tar.gz

In [ ]:
from os import listdir

from PIL import Image as PImage, ImageDraw as PImageDraw

from utils.detect_utils import OBJECT_THRESHOLDS
from models.Detr import Detr
from models.Dino import Dino
from models.Owlv2 import Owlv2
from models.Yolo import Yolo

In [ ]:
IMG_DIR = "./imgs/address"
imgs = [PImage.open(f"{IMG_DIR}/{f}") for f in sorted(listdir(IMG_DIR)) if f.endswith("jpg")]

In [ ]:
def visualize_objs(img, objs):
  dimg = img.copy()
  iw,ih = dimg.size
  draw = PImageDraw.Draw(dimg)

  for o in objs:
    x0,y0,x1,y1 = o["box"]
    draw.rectangle([x0*iw,y0*ih,x1*iw,y1*ih], outline=(0,255,0), width=4)

  display(dimg.resize((256, 256)))
  print([f'{o["label"]}: {o["score"]}' for o in objs], '\n')

In [ ]:
model = Yolo()

for img in imgs:
  objs = model.all_objects(img, OBJECT_THRESHOLDS)
  visualize_objs(img, objs)

In [ ]:
model = Owlv2()

for img in imgs:
  objs = model.iou_objects(img, OBJECT_THRESHOLDS)
  visualize_objs(img, objs)

In [ ]:
model = Detr()

for img in imgs:
  objs = model.all_objects(img, OBJECT_THRESHOLDS)
  visualize_objs(img, objs)

In [ ]:
model = Dino()

for img in imgs:
  objs = model.iou_objects(img, OBJECT_THRESHOLDS)
  visualize_objs(img, objs)

## OCR:
- https://huggingface.co/blog/ocr-open-models
- [Qwen3-VL](https://huggingface.co/collections/Qwen/qwen3-vl)
- [Chandra](https://huggingface.co/datalab-to/chandra) (optimized for docs) (out of memory)
- [olmOCR](https://huggingface.co/allenai/olmOCR-2-7B-1025) (documents)
- [PaddleOCR](https://huggingface.co/PaddlePaddle/PaddleOCR-VL) (documents)
- [Nanonets-OCR2](https://huggingface.co/nanonets/Nanonets-OCR2-3B) (documents)
- [DeepSeek-OCR](https://huggingface.co/deepseek-ai/DeepSeek-OCR) (won't install `flash-attn` dependency)
- [TrOCR] [[scene](https://huggingface.co/microsoft/trocr-large-str) > [handwritten](https://huggingface.co/microsoft/trocr-large-handwritten) > [printed](https://huggingface.co/microsoft/trocr-large-printed)] (requires pre-crop, maybe with EasyOCR)
- [EasyOCR](https://github.com/JaidedAI/EasyOCR) 🤔
- [Easy-Yolo-OCR](https://github.com/aqntks/Easy-Yolo-OCR)

In [ ]:
from os import listdir

from PIL import Image as PImage, ImageDraw as PImageDraw

In [ ]:
IMG_DIR = "./imgs/address"
img_names = [1001, 1013, 1014, 1025, 102]
# imgs = [PImage.open(f"{IMG_DIR}/{f}") for f in sorted(listdir(IMG_DIR))[:100] if f.endswith("jpg")]
imgs = [PImage.open(f"{IMG_DIR}/{f}.jpg") for f in img_names]

imgs[2]

### EasyOCR

In [ ]:
!pip install easyocr

In [ ]:
import easyocr
reader = easyocr.Reader(['pt', 'en'])

In [ ]:
mimg = imgs[2]
mdraw = PImageDraw.Draw(mimg)

result = reader.readtext(f'./imgs/address/{img_names[2]}.jpg')

In [ ]:
for b,t,_ in result:
  mdraw.rectangle((b[0], b[2]), width=2)

mimg

### Chandra

In [ ]:
!pip install chandra-ocr

In [ ]:
from transformers import AutoModel, AutoProcessor
from chandra.model.hf import generate_hf
from chandra.model.schema import BatchInputItem
from chandra.output import parse_markdown

model = AutoModel.from_pretrained("datalab-to/chandra").to("cuda")
model.processor = AutoProcessor.from_pretrained("datalab-to/chandra")

batch = [
  BatchInputItem(
    image=imgs[2],
    prompt_type="ocr"
  )
]

result = generate_hf(batch, model)
# markdown = parse_markdown(result.raw)

### TrOCR

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-str')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-str').to("cuda")
pixel_values = processor(images=imgs[2], return_tensors="pt").pixel_values.to("cuda")

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)

In [ ]:
display(imgs[2])
generated_text

### EasyOCR + TrOCR

In [ ]:
import easyocr

from transformers import TrOCRProcessor, VisionEncoderDecoderModel

reader = easyocr.Reader(["pt", "en"])

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-str")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-str").to("cuda")

In [ ]:
mimg = imgs[2]
result = reader.readtext(f"./imgs/address/{img_names[2]}.jpg")

for b,t,s in result:
  (x0, y0), (x1, y1) = b[0], b[2]
  ba = (x1 - x0) * (y1 - y0)
  if ba > 500 and s > 0.02:
    pixel_values = processor(images=mimg.crop((x0,y0,x1,y1)), return_tensors="pt").pixel_values.to("cuda")
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)

In [ ]:
from models.Ocr import Ocr

In [ ]:
IMG_DIR = "./imgs/address"
img_paths = [f"{IMG_DIR}/{f}" for f in sorted(listdir(IMG_DIR)) if f.endswith("jpg")]

ocr = Ocr()

for path in img_paths:
  words = ocr.top_words(path, 500, 0.02)
  print(words)